In [38]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [39]:
titanic_df = pd.read_csv('data/titanic_processed.csv')
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,1,15.0,1,1,7.2292,0,0
1,0,3,1,22.0,0,0,7.2500,0,1
2,0,2,1,18.0,0,0,13.0000,0,1
3,1,2,0,18.0,0,1,23.0000,0,1
4,0,3,1,20.0,0,0,7.8542,0,1


In [40]:
features = list(titanic_df.columns[1:])
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked_Q', 'Embarked_S']

In [41]:
results_dict = {}

In [42]:
def summarize_classification(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred, normalize = True)
    num_acc = accuracy_score(y_test, y_pred, normalize = False)
    
    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    
    return {'accuracy' : acc,
            'precision' : prec, 
            'recall' : recall,
            'accurcy_count' : num_acc     
            }

In [55]:
def build_model(classifier_fn, name_of_y_col, names_of_x_cols, dataset, test_frac = 0.2):
    
    x = dataset[names_of_x_cols]
    y = dataset[name_of_y_col]
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = test_frac)
    
    model = classifier_fn(x_train, y_train)
    y_pred = model.predict(x_test)
    
    y_pred_train = model.predict(x_train)
    
    train_summary = summarize_classification(y_train, y_pred_train)
    test_summary = summarize_classification(y_test, y_pred)
    
    pred_results = pd.DataFrame({'y_test': y_test,
                                 'y_pred': y_pred,
                                })
    
    # This is the confusionmatrix
    model_crosstab = pd.crosstab(pred_results.y_test, pred_results.y_pred)
    
    return {'training' : train_summary,
            'test': test_summary,
            'confusion_matrix' : model_crosstab
           }

In [73]:
def compare_results():
    for keys in results_dict:
        print({'classification ' : keys})
        
    print()
    print('Training Data')
    for score in results_dict[keys]['training']:
        print(score, results_dict[keys]['training'][score])
        
        
    print()
    print('Test Data')
    for score in results_dict[keys]['test']:
        print(score, results_dict[keys]['test'][score])
        
        

In [74]:
def logistic_fn(x_train, y_train):
    model = LogisticRegression(solver = 'liblinear')
    model.fit(x_train,y_train)
    
    return model

In [75]:
results_dict['survived - logistic'] = build_model(logistic_fn, 'Survived', features, titanic_df)

compare_results()

{'classification ': 'survived - logistic'}

Training Data
accuracy 0.7996485061511424
precision 0.788659793814433
recall 0.6769911504424779
accurcy_count 455

Test Data
accuracy 0.8181818181818182
precision 0.8461538461538461
recall 0.7096774193548387
accurcy_count 117


In [ ]:
def linear_discriminant_fn(x_train, y_train, solver ='svd'):
    model = 